In [10]:
import string

In [11]:
ascii_characters = string.printable
ascii_characters 

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [15]:
ascii_characters = ascii_characters[:-14]
ascii_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\'

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()
driver.get('https://onlineasciitools.com/convert-ascii-to-image')


driver.execute_script('window.scrollTo(0, 900);')
font_dropdown = driver.find_element(by=By.CSS_SELECTOR, value='#option-typeface-9b9bc56c')


/html/body/div[1]/div[2]/div[2]/div[3]/div/div[4]/div[2]/div/div[2]/div[3]/div/select

font_dropdown.click()

actions.move_to_element('window.scrollTo(0, 0);').perform()               

input_box = driver.find_element_by_css_selector(
    'body > div.page-content > div.page.container > div.content > div.all-tools-container > div > div.body > '
    'div.sides-wrapper.clearfix > div.side.input > div > div.side-wrapper > div.data-wrapper > textarea'
)

#for character in ascii_characters:
#    input_box.send_keys()
#    input_box.clear()

driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#option-typeface-9b9bc56c"}
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x0065FDC3+2555331]
	Ordinal0 [0x005F77F1+2127857]
	Ordinal0 [0x004F2E08+1060360]
	Ordinal0 [0x0051E49E+1238174]
	Ordinal0 [0x0051E69B+1238683]
	Ordinal0 [0x00549252+1413714]
	Ordinal0 [0x00537B54+1342292]
	Ordinal0 [0x005475FA+1406458]
	Ordinal0 [0x00537976+1341814]
	Ordinal0 [0x005136B6+1193654]
	Ordinal0 [0x00514546+1197382]
	GetHandleVerifier [0x007F9622+1619522]
	GetHandleVerifier [0x008A882C+2336844]
	GetHandleVerifier [0x006F23E1+541697]
	GetHandleVerifier [0x006F1443+537699]
	Ordinal0 [0x005FD18E+2150798]
	Ordinal0 [0x00601518+2168088]
	Ordinal0 [0x00601660+2168416]
	Ordinal0 [0x0060B330+2208560]
	BaseThreadInitThunk [0x763F6739+25]
	RtlGetFullPathName_UEx [0x776F8AFF+1215]
	RtlGetFullPathName_UEx [0x776F8ACD+1165]
